In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pathlib

import duckdb
import pandas as pd

from autoanalyst import entity, transform, tree

# In-memory DB for quick work (change to 'mydb.duckdb' to persist)
DB_PATH = pathlib.Path("adventureworks.duckdb")

sales_cols = [
    "buyer_count",
    "active_days",
    "order_count",
    "order_lines",
    "order_quantity",
    "order_sales",
    "cost_production",
    "cost_tax",
    "cost_freight",
    "cost_discount",
    "weight_synthetic",
    "size_synthetic",
]

customer_cols = [
    "customer_id",
    "marital_status",
    "house_owner",
    "commute_distance",
    "cars_category",
    "children_category",
    "income_category",
    "country_region_code",
    "state_province_code",
    "city",
    "sales_territory",
    "activity_category",
    "customer_age_category",
]

reseller_cols = [
    "reseller_id",
    "business_type",
    "order_frequency",
    "product_line",
    "annual_sales",
    "annual_revenue",
    "country_region_code",
    "state_province_code",
    "city",
    "sales_territory",
    "activity_category",
]

business_unit_cols = [
    "report_date",
    "department_name",
]

opex_cols = [
    "report_date",
    "department_name",
    "headcount",
    "salary_cost",
    "benefits_cost",
    "overtime_cost",
    "training_cost",
    "travel_cost",
    "misc_labor_cost",
    "go_to_market_cost",
    "facilities_cost",
    "it_infra_cost",
    "fees_prof_services_cost",
    "misc_nonlabor_cost",
]

cogs_cols = [
    "cost_production",
    "cost_tax",
    "cost_freight",
    "cost_discount",
]


def read_query(query):
    """
    Read data from the DuckDB database.
    """
    with duckdb.connect(DB_PATH) as conn:
        return conn.execute(query).df()


internet_query = """
    select * from int__internetsales_orders
    order by customer_id, report_date
"""
reseller_query = """
    select * from int__resellersales_orders
    order by reseller_id, report_date
"""
business_unit_opex = """
    select * from int__department_costings
    where department_name != 'Corporate'
    order by department_name, report_date
"""

df_internet_sales = read_query(internet_query)
df_reseller_sales = read_query(reseller_query)
df_opex = read_query(business_unit_opex)


df_buyer_internet = df_internet_sales[customer_cols]
df_buyer_resale = df_reseller_sales[reseller_cols]

df_internet_sales[sales_cols].head()
# df_buyer_internet.head()

,buyer_count,active_days,order_count,order_lines,order_quantity,order_sales,cost_production,cost_tax,cost_freight,cost_discount,weight_synthetic,size_synthetic
0,1,1,1,1,1.0,3399.99,-1912.1544,-271.9992,-84.9998,-0.0,2500.0,15000.0
1,1,0,0,0,0.0,0.00,-0.0000,-0.0000,-0.0000,-0.0,0.0,0.0
2,0,0,0,0,0.0,0.00,-0.0000,-0.0000,-0.0000,-0.0,0.0,0.0
3,0,0,0,0,0.0,0.00,-0.0000,-0.0000,-0.0000,-0.0,0.0,0.0
4,0,0,0,0,0.0,0.00,-0.0000,-0.0000,-0.0000,-0.0,0.0,0.0


In [ ]:
def process_sales_data(df, id_col, date_col):
    """
    Process sales data for metric demcomp
    """
    out = df[[id_col, date_col]].copy()

    out["buyer_count"] = df.buyer_count

    out["active_days_per_buyer"] = df.active_days / df.buyer_count
    out["order_count_per_active_day"] = df.order_count / df.active_days
    out["order_lines_per_order"] = df.order_lines / df.order_count
    out["order_units_per_line"] = df.order_quantity / df.order_lines
    out["order_sales_per_unit"] = df.order_sales / df.order_quantity

    total_costs = df[cogs_cols].sum(axis=1)

    out["cogs_profit_deflator"] = 1 + (total_costs / df.order_sales)

    cost_allocation_block = df[cogs_cols].copy().div(total_costs.values, axis=0)
    out[cogs_cols] = cost_allocation_block.mul(out.cogs_profit_deflator, axis=0)

    out["per_unit_weight"] = df.weight_synthetic / df.order_quantity
    out["per_unit_size"] = df.size_synthetic / df.order_quantity

    out["net_profit"] = df.order_sales + total_costs
    return out.fillna(0)


def process_opex_data(df, id_col="department_name", date_col="report_date"):
    """
    Process OPEX data for metric demcomp
    """
    out = df[[id_col, date_col, "headcount"]].copy()
    total_labor_costs = (
        df.salary_cost
        + df.benefits_cost
        + df.overtime_cost
        + df.training_cost
        + df.travel_cost
        + df.misc_labor_cost
    )
    total_nonlabor_cost = (
        df.go_to_market_cost
        + df.facilities_cost
        + df.it_infra_cost
        + df.fees_prof_services_cost
        + df.misc_nonlabor_cost
    )
    headcount = df.headcount.fillna(0)

    out["per_employee_labor_cost"] = total_labor_costs / headcount
    out["per_employee_salary_cost"] = df.salary_cost / headcount
    out["per_employee_benefits_cost"] = df.benefits_cost / headcount
    out["per_employee_overtime_cost"] = df.overtime_cost / headcount
    out["per_employee_training_cost"] = df.training_cost / headcount
    out["per_employee_travel_cost"] = df.travel_cost / headcount

    out["go_to_market_cost"] = df.go_to_market_cost
    out["facilities_cost"] = df.facilities_cost
    out["it_infra_cost"] = df.it_infra_cost
    out["fees_prof_services_cost"] = df.fees_prof_services_cost

    out["total_opex"] = total_labor_costs + total_nonlabor_cost
    out["total_nonlabor_cost"] = total_nonlabor_cost
    out["total_per_employee_labor_cost"] = total_labor_costs / headcount

    return out.fillna(0)


df_internet_sales_cleaned = process_sales_data(
    df_internet_sales,
    id_col="customer_id",
    date_col="report_date",
)

df_reseller_sales_cleaned = process_sales_data(
    df_reseller_sales,
    id_col="reseller_id",
    date_col="report_date",
)

df_opex_cleaned = process_opex_data(
    df_opex,
    id_col="department_name",
    date_col="report_date",
)

df_internet_sales_cleaned.head()

df_company = pd.concat(
    [
        df_internet_sales_cleaned.groupby("report_date")
        .net_profit.sum()
        .rename("internet_net_profit"),
        df_reseller_sales_cleaned.groupby("report_date")
        .net_profit.sum()
        .rename("reseller_net_profit"),
        df_opex_cleaned.groupby("report_date").total_opex.sum().rename("total_opex"),
    ],
    axis=1,
).reset_index()
df_company["net_position"] = (
    df_company.internet_net_profit
    + df_company.reseller_net_profit
    + df_company.total_opex
)
df_company["company_id"] = 1

In [9]:
from autoanalyst.core.base_classes import BaseEntity

company_topline = tree.MetricTreeNode(
    name="company_topline",
    entity="company",
    head_col="net_position",
    children_cols=["reseller_net_profit", "internet_net_profit", "total_opex"],
    parent_node_lookup=None,  # parent node is the root
    transform=transform.SubtotalTransform(),
)

internet_profit = tree.MetricTreeNode(
    name="internet_profit",
    entity="internet_customer",
    head_col="net_profit",
    children_cols=[
        "buyer_count",
        "active_days_per_buyer",
        "order_count_per_active_day",
        "order_lines_per_order",
        "order_units_per_line",
        "order_sales_per_unit",
        "cogs_profit_deflator",
    ],
    parent_node_lookup=tree.ParentColLink(
        node_name="company_topline",
        child_col="internet_net_profit",
    ),
    transform=transform.MetricDecompTransform(),
)

company_entity = entity.DataFrameEntity(
    name="company",
    df=df_company,
    id_col="company_id",
    date_col="report_date",
)

internet_customer_entity = entity.DataFrameEntity(
    name="internet_customer",
    df=df_internet_sales_cleaned,
    id_col="customer_id",
    date_col="report_date",
)

tree_nodes = [
    company_topline,
    internet_profit,
]

tree_entities: list[BaseEntity] = [
    company_entity,
    internet_customer_entity,
]

metric_tree = tree.MetricTree(
    name="adventureworks_tree", entities=tree_entities, nodes=tree_nodes
)